# Topic Model



In [1]:
# 必要パッケージのインポートと不思議なおまじない

import matplotlib.pyplot as plt
import numpy as np
# np.random.seed(0)
import sklearn as sl
import pandas as pd
import seaborn as sns
import scipy as sp
import gensim

%matplotlib inline

## LSAとLDAを比較してみる

### 人工データの作成

In [2]:
# 5つの文書
documents = [
    {'drive': 3, 'automobile': 3},  # 車
    {'drive': 4, 'car': 4},  # 車
    {'play': 2, 'music': 2},  # 音楽
    {'play': 3, 'guitar': 2},  # 音楽
    {'drive': 1, 'car': 1, 'music': 1},  # 車と音楽
]

documents = [
    [word for word, count in doc.items() for _ in range(count)]
    for doc in documents
]

documents

[['drive', 'drive', 'drive', 'automobile', 'automobile', 'automobile'],
 ['drive', 'drive', 'drive', 'drive', 'car', 'car', 'car', 'car'],
 ['play', 'play', 'music', 'music'],
 ['play', 'play', 'play', 'guitar', 'guitar'],
 ['drive', 'car', 'music']]

In [3]:
dictionary = gensim.corpora.Dictionary(documents)
dict(dictionary)

{0: 'automobile', 1: 'drive', 2: 'car', 3: 'music', 4: 'play', 5: 'guitar'}

In [4]:
corpus_bow = [dictionary.doc2bow(doc) for doc in documents]
corpus_bow

[[(0, 3), (1, 3)],
 [(1, 4), (2, 4)],
 [(3, 2), (4, 2)],
 [(4, 3), (5, 2)],
 [(1, 1), (2, 1), (3, 1)]]

### Latent Semantic Analysis

重みが正負どちらもあり得ることを確認する

In [5]:
model_lsi = gensim.models.LsiModel(corpus_bow, id2word=dictionary, num_topics=2)
model_lsi.print_topics()

[(0,
  '0.791*"drive" + 0.567*"car" + 0.224*"automobile" + 0.039*"music" + 0.006*"play" + 0.001*"guitar"'),
 (1,
  '0.870*"play" + 0.399*"guitar" + 0.287*"music" + -0.019*"automobile" + -0.017*"drive" + 0.002*"car"')]

- トピック0: 車っぽい
- トピック1: 音楽っぽい

`car`と`automobile`は一度も共起していないのに同じようにまとめられた！

In [6]:
(0.791 * -0.017) + (0.567 * 0.002) + (0.224 * -0.019) + (0.039 * 0.287) + (0.006 * 0.870) + (0.001 * 0.399)

0.00024299999999999582

内積がほぼ0 => ベクトルが直交

In [7]:
corpus_lsi = model_lsi[corpus_bow]
list(corpus_lsi)

[[(0, 3.046659340668715), (1, -0.10709119088243702)],
 [(0, 5.435106071663031), (1, -0.05976833382093923)],
 [(0, 0.08875793440700527), (1, 2.314765187142154)],
 [(0, 0.019167790544852435), (1, 3.4096947425844277)],
 [(0, 1.3973928739763606), (1, 0.2721550444656107)]]

- 文書0: ほとんど車（トピック0）
- 文書1: ほとんど車（トピック0）
- 文書2: ほとんど音楽（トピック1）
- 文書3: ほとんど音楽（トピック1）
- 文書4: 多めの車（トピック0）と少なめの音楽（トピック1）

### Latent Dirichlet Allocation

重みが非負（確率）であることを確認する

In [8]:
# random_stateに定数を指定して計算し直しても同じ答えが得られるように
model_lda = gensim.models.LdaModel(corpus_bow, id2word=dictionary, num_topics=2, random_state=0)
model_lda.print_topics()

[(0,
  '0.407*"drive" + 0.265*"car" + 0.176*"automobile" + 0.060*"music" + 0.050*"play" + 0.041*"guitar"'),
 (1,
  '0.370*"play" + 0.211*"music" + 0.164*"guitar" + 0.114*"drive" + 0.085*"car" + 0.057*"automobile"')]

- トピック0: 車っぽい
- トピック1: 音楽っぽい

`car`と`automobile`は一度も共起していないのに同じようにまとめられた

In [9]:
0.407 + 0.265 + 0.176 + 0.060 + 0.050 + 0.041

0.999

In [10]:
0.370 + 0.211 + 0.164 + 0.114 + 0.085 + 0.057

1.001

トピック・単語分布: 各値は非負で合計は（ほぼ）1

In [11]:
corpus_lda = model_lda[corpus_bow]
list(corpus_lda)

[[(0, 0.923971), (1, 0.076028995)],
 [(0, 0.94044226), (1, 0.05955771)],
 [(0, 0.1043772), (1, 0.8956228)],
 [(0, 0.085959926), (1, 0.91404)],
 [(0, 0.67152566), (1, 0.32847437)]]

- 文書0: ほとんど車（トピック0）
- 文書1: ほとんど車（トピック0）
- 文書2: ほとんど音楽（トピック1）
- 文書3: ほとんど音楽（トピック1）
- 文書4: 多めの車（トピック0）と少なめの音楽（トピック1）
    
文書・トピック分布: 各値は非負で合計は（ほぼ）1

### 検索

In [12]:
documents

[['drive', 'drive', 'drive', 'automobile', 'automobile', 'automobile'],
 ['drive', 'drive', 'drive', 'drive', 'car', 'car', 'car', 'car'],
 ['play', 'play', 'music', 'music'],
 ['play', 'play', 'play', 'guitar', 'guitar'],
 ['drive', 'car', 'music']]

In [13]:
corpus_bow

[[(0, 3), (1, 3)],
 [(1, 4), (2, 4)],
 [(3, 2), (4, 2)],
 [(4, 3), (5, 2)],
 [(1, 1), (2, 1), (3, 1)]]

In [14]:
query = ['car']
query_bow = dictionary.doc2bow(query)
query_bow

[(2, 1)]

In [15]:
sim_bow = gensim.similarities.MatrixSimilarity(corpus_bow)
sim_bow[query_bow]

array([0.        , 0.70710677, 0.        , 0.        , 0.57735026],
      dtype=float32)

車関係だけど`car`が出現しない文書0は検索に引っかからない

In [16]:
query_lsi = model_lsi[query_bow]
query_lsi

[(0, 0.5673333074250925), (1, 0.001937632239003192)]

In [17]:
sim_lsi = gensim.similarities.MatrixSimilarity(corpus_lsi)
sim_lsi[query_lsi]

array([0.99925697, 0.99989617, 0.04172867, 0.00903669, 0.9822047 ],
      dtype=float32)

車関係だけど`car`が出現しない文書0も検索に引っかかる

In [18]:
query_lda = model_lda[query_bow]
query_lda

[(0, 0.7322643), (1, 0.26773572)]

In [19]:
sim_lda = gensim.similarities.MatrixSimilarity(corpus_lda)
sim_lda[query_lda]

array([0.96418893, 0.9590184 , 0.44981128, 0.42982286, 0.9945636 ],
      dtype=float32)

車関係だけど`car`が出現しない文書0も検索に引っかかる

## 実データにLDAを適用してみる

Wikipediaから適当な記事を選択して1つ目の段落を利用

In [20]:
documents = [
    # Topic model
    '''In machine learning and natural language processing, a topic model is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents. Topic modeling is a frequently used text-mining tool for discovery of hidden semantic structures in a text body. Intuitively, given that a document is about a particular topic, one would expect particular words to appear in the document more or less frequently: "dog" and "bone" will appear more often in documents about dogs, "cat" and "meow" will appear in documents about cats, and "the" and "is" will appear equally in both. A document typically concerns multiple topics in different proportions; thus, in a document that is 10% about cats and 90% about dogs, there would probably be about 9 times more dog words than cat words. The "topics" produced by topic modeling techniques are clusters of similar words. A topic model captures this intuition in a mathematical framework, which allows examining a set of documents and discovering, based on the statistics of the words in each, what the topics might be and what each document's balance of topics is.''',
    # Latent Dirichlet allocation
    '''In natural language processing, latent Dirichlet allocation (LDA) is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar. For example, if observations are words collected into documents, it posits that each document is a mixture of a small number of topics and that each word's presence is attributable to one of the document's topics. LDA is an example of a topic model.''',
    # Probabilistic latent semantic analysis
    '''Probabilistic latent semantic analysis (PLSA), also known as probabilistic latent semantic indexing (PLSI, especially in information retrieval circles) is a statistical technique for the analysis of two-mode and co-occurrence data. In effect, one can derive a low-dimensional representation of the observed variables in terms of their affinity to certain hidden variables, just as in latent semantic analysis, from which PLSA evolved.''',
    
    # WBSC Premier12
    '''The WBSC Premier12 is the flagship international baseball tournament organized by the World Baseball Softball Confederation (WBSC), featuring the 12 highest-ranked national baseball teams in the world. The first tournament was held in Taiwan and Japan in November 2015. The second tournament, 2019 WBSC Premier12, is being held in November 2019.''',
    # World Baseball Classic
    '''The World Baseball Classic (WBC) is an international baseball tournament sanctioned from 2006 to 2013 by the International Baseball Federation (IBAF) and after 2013 by the World Baseball Softball Confederation (WBSC). It was proposed to the IBAF by Major League Baseball (MLB), the Major League Baseball Players Association (MLBPA), and other professional baseball leagues and their players associations around the world. It is the main baseball tournament sanctioned by the WBSC, which grants to the winner the title of "World Champion".[1]''',
    # Baseball at the 2020 Summer Olympics
    '''Baseball will be featured at the 2020 Summer Olympics, in Tokyo, for the first time since the 2008 Summer Olympics.[1] Six national teams will compete in the tournament, with Japan (host), Israel, South Korea, and Mexico having qualified so far. Baseball/softball is one of five sports that were added to the program of the 2020 Summer Olympics only, with no guarantee it will return in 2024.''',
    
    # Google
    '''Google LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, search engine, cloud computing, software, and hardware. It is considered one of the Big Four technology companies, alongside Amazon, Apple, and Facebook.[10][11]''',
    # Amazon (company)
    '''Amazon.com, Inc.[6] (/ˈæməzɒn/), is an American multinational technology company based in Seattle, Washington, that focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is considered one of the Big Four technology companies along with Google, Apple, and Facebook.[7][8][9]''',
    # Facebook
    '''Facebook, Inc. is an American online social media and social networking service company based in Menlo Park, California. It was founded by Mark Zuckerberg, along with fellow Harvard College students and roommates Eduardo Saverin, Andrew McCollum, Dustin Moskovitz and Chris Hughes. It is considered one of the Big Four technology companies along with Amazon, Apple, and Google.[12][13]''',
    # Apple Inc.
    '''Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services. It is considered one of the Big Four tech companies along with Amazon, Google, and Facebook.[6][7]''',
]

In [21]:
documents = [
    [word for word in doc.lower().split()]
    for doc in documents
]

In [22]:
dictionary = gensim.corpora.Dictionary(documents)
len(dictionary)

350

In [23]:
dictionary.filter_extremes(no_below=2, no_above=0.3)
len(dictionary)

47

In [24]:
corpus_bow = [dictionary.doc2bow(doc) for doc in documents]

In [25]:
model_lda = gensim.models.LdaModel(corpus_bow, id2word=dictionary, num_topics=3, random_state=0)
model_lda.print_topics()

[(0,
  '0.191*"a" + 0.055*"document" + 0.055*"topic" + 0.045*"model" + 0.045*"topics" + 0.045*"words" + 0.045*"semantic" + 0.045*"latent" + 0.035*"will" + 0.035*"be"'),
 (1,
  '0.174*"baseball" + 0.055*"will" + 0.052*"tournament" + 0.046*"world" + 0.045*"international" + 0.030*"national" + 0.030*"confederation" + 0.029*"tournament," + 0.029*"softball" + 0.028*"world."'),
 (2,
  '0.083*"along" + 0.061*"companies" + 0.046*"based" + 0.045*"apple," + 0.044*"inc." + 0.044*"online" + 0.044*"baseball" + 0.041*"google," + 0.041*"amazon," + 0.040*"multinational"')]

- トピック0: トピックモデル
- トピック1: 野球
- トピック2: GAFA

In [26]:
corpus_lda = model_lda[corpus_bow]
list(corpus_lda)

[[(0, 0.98553497)],
 [(0, 0.969543), (1, 0.015245173), (2, 0.015211832)],
 [(0, 0.94744855), (1, 0.026765607), (2, 0.025785843)],
 [(0, 0.019654764), (1, 0.95912224), (2, 0.021223053)],
 [(0, 0.016030397), (1, 0.9672503), (2, 0.01671929)],
 [(0, 0.032354172), (1, 0.93595755), (2, 0.031688277)],
 [(0, 0.041971438), (1, 0.047950782), (2, 0.91007775)],
 [(0, 0.033979114), (1, 0.035012007), (2, 0.9310089)],
 [(0, 0.033558454), (1, 0.034653597), (2, 0.93178797)],
 [(0, 0.037186656), (1, 0.03946741), (2, 0.9233459)]]

それっぽくなった